# License Plate Detection Model for Mobia

The following  notebook is based on the NVIDIA Developer Technical Blog [Creating a Real-Time License Plate Detection and Recognition App](https://developer.nvidia.com/blog/creating-a-real-time-license-plate-detection-and-recognition-app/) by [Yue Zhu](https://developer.nvidia.com/blog/author/tylerz/), [Morgan Huang](https://developer.nvidia.com/blog/author/morganh/) and [Fei Chen](https://developer.nvidia.com/blog/author/fechen/).

Transfer learning is the process of transferring learned features from one application to another. It is a commonly used training technique where you use a model trained on one task and re-train to use it on a different task. 
Train Adapt Optimize (TAO) Toolkit  is a simple and easy-to-use Python based AI toolkit for taking purpose-built AI models and customizing them with users' own data.

<img align="center" src="https://developer.nvidia.com/sites/default/files/akamai/embedded-transfer-learning-toolkit-software-stack-1200x670px.png" width="1080"> 

## Learning Objectives
In this notebook, you will learn how to leverage the simplicity and convenience of TAO to:

* Take a pretrained resnet18 model and train a ResNet-18 DetectNet_v2 model on the KITTI dataset
* Prune the trained detectnet_v2 model
* Retrain the pruned model to recover lost accuracy
* Export the pruned model
* Quantize the pruned model using QAT
* Run Inference on the trained model
* Export the pruned, quantized and retrained model to a .etlt file for deployment to DeepStream
* Run inference on the exported. etlt model to verify deployment using TensorRT

### Table of Contents

This notebook shows an example usecase of Object Detection using DetectNet_v2 in the Train Adapt Optimize (TAO) Toolkit.

0. [Set up env variables and map drives](#head-0)
1. [Install the requirements and the TAO launcher](#head-1)
    1. [Install Anaconda](#head-1-1)
    1. [Install NVIDIA Container Toolkit](#head-1-2)
    2. [Install Tao Launcher](#head-1-3)
1. [Prepare dataset and pre-trained model](#head-2)
    1. [Download the dataset](#head-2-1)
    1. [Resize images and labels](#head-2-2)
    1. [Split data](#head-2-3)
    2. [Download pre-trained model](#head-2-4)
2. [Provide training specification](#head-3)
3. [Run TAO training](#head-4)
4. [Evaluate trained models](#head-5)
5. [Prune trained models](#head-6)
6. [Retrain pruned models](#head-7)
7. [Evaluate retrained model](#head-8)
8. [Visualize inferences](#head-9)
9. [Model Export](#head-10)
    1. [Int8 Optimization](#head-10-1)
    2. [Generate TensorRT engine](#head-10-2)
10. [Verify Deployed Model](#head-11)
    1. [Inference using TensorRT engine](#head-11-1)
11. [QAT workflow](#head-12)
    1. [Convert pruned model to QAT and retrain](#head-12-1)
    2. [Evaluate QAT converted model](#head-12-2)
    3. [Export QAT trained model to int8](#head-12-3)
    4. [Evaluate a QAT trained model using the exported TensorRT engine](#head-12-4)
    5. [Inference using QAT engine](#head-12-5)

## 0. Set up env variables and map drives <a class="anchor" id="head-0"></a>

To start with this notebook we have to set local and Tao container environment variables. To do that, we use a same-name folder distribution tree for local machine and Tao Container. We will used the prefix `LOCAL` for local directories and `INT` for directories from Tao container. In the following cell, the environment variables are setting up. 

When using the purpose-built pretrained models from NGC, please make sure to set the `$KEY` environment variable to the key as mentioned in the model overview. Failing to do so, can lead to errors when trying to load them as pretrained models.

### Project Directory Tree for the License Plate Detection (LPDNet)

```
└── lpd_net
    ├── data
    |   ├── benchmark
    |   ├── lpd
    |   └── lpd_tfrecord
    ├── experiments
    |   ├── experiment_test
    |   └── lpdnet_vunpruned_v1.0
    ├── specs
    └── lpd_prepare_data.py
```

Please note that the dataset to run this notebook is expected to reside in the `$LOCAL_PROJECT_DIR/data`, while the TAO experiment generated collaterals will be output to `$LOCAL_PROJECT_DIR/experiments`. More information on how to set up the dataset and the supported steps in the TAO workflow are provided in the subsequent cells.

*Note: Please make sure to remove any stray artifacts/files from the `$INT_MAIN_DIR` or `$INT_DATA_DIR` paths as mentioned below, that may have been generated from previous experiments. Having checkpoint files etc may interfere with creating a training graph for a new experiment.*

*Note: This notebook currently is by default set up to run training using 1 GPU. To use more GPU's please update the env variable `$NUM_GPUS` accordingly*

In [1]:
# Setting up env variables for cleaner command line commands.
import os

!printf "Environment Variables:\n\n"
%env KEY=tlt_encode
%env NUM_GPUS=1
!printf "\nInternal Paths for Tao Workspace:\n\n" 
%env INT_MAIN_DIR=/workspace/tao-experiments/lpd_net
%env INT_DATA_DIR=/workspace/tao-experiments/lpd_net/data
%env INT_SPECS_DIR=/workspace/tao-experiments/lpd_net/specs

# Set this path if you don't run the notebook from the samples directory.
# %env NOTEBOOK_ROOT=~/tao-samples/detectnet_v2

# Please define this local project directory that needs to be mapped to the TAO docker session.
# The dataset expected to be present in $LOCAL_PROJECT_DIR/data, while the results for the steps
# in this notebook will be stored at $LOCAL_PROJECT_DIR/detectnet_v2
# !PLEASE MAKE SURE TO UPDATE THIS PATH!.

os.environ["LOCAL_PROJECT_DIR"] = os.getcwd()

os.environ["LOCAL_EXPERIMENT_DIR"] = os.path.join(
    os.getenv("LOCAL_PROJECT_DIR", os.getcwd()),
    "experiments"
)

# The sample spec files are present in the same path as the downloaded samples.
os.environ["LOCAL_SPECS_DIR"] = os.path.join(
    os.getenv("NOTEBOOK_ROOT", os.getcwd()),
    "specs"
)

os.environ["LOCAL_DATA_DIR"] = os.path.join(
    os.getenv("NOTEBOOK_ROOT", os.getcwd()),
    "data"
)

# Showing list of specification files.
# !ls -rlt $LOCAL_SPECS_DIR
!printf "\nPaths for local machine:\n\n" 
!echo LOCAL_PROJECT_DIR:    $LOCAL_PROJECT_DIR
!echo LOCAL_DATA_DIR:       $LOCAL_DATA_DIR
!echo LOCAL_EXPERIMENT_DIR: $LOCAL_EXPERIMENT_DIR
!echo LOCAL_SPECS_DIR:      $LOCAL_SPECS_DIR

Environment Variables:

env: KEY=tlt_encode
env: NUM_GPUS=1

Internal Paths for Tao Workspace:

env: INT_MAIN_DIR=/workspace/tao-experiments/lpd_net
env: INT_DATA_DIR=/workspace/tao-experiments/lpd_net/data
env: INT_SPECS_DIR=/workspace/tao-experiments/lpd_net/specs

Paths for local machine:

LOCAL_PROJECT_DIR: /home/pepeleduin/mobia/mobia-project/training/lpd_net
LOCAL_DATA_DIR: /home/pepeleduin/mobia/mobia-project/training/lpd_net/data
LOCAL_EXPERIMENT_DIR: /home/pepeleduin/mobia/mobia-project/training/lpd_net/experiments
LOCAL_SPECS_DIR: /home/pepeleduin/mobia/mobia-project/training/lpd_net/specs


The cell below maps the project directory on your local host to a workspace directory in the TAO docker instance, so that the data and the results are mapped from in and out of the docker. For more information please refer to the [launcher instance](https://docs.nvidia.com/tao/tao-toolkit/tao_launcher.html) in the user guide.

When running this cell on AWS, update the drive_map entry with the dictionary defined below, so that you don't have permission issues when writing data into folders created by the TAO docker.

In [2]:
# Mapping up the local directories to the TAO docker.
import json
mounts_file = os.path.expanduser("~/.tao_mounts.json")

# Define the dictionary with the mapped drives
drive_map = {
    "Mounts": [
        # Mapping the data directory
        {
            "source": os.environ["LOCAL_PROJECT_DIR"],
            "destination": os.environ["INT_MAIN_DIR"]
        },
        # Mapping the specs directory.
        {
            "source": os.environ["LOCAL_SPECS_DIR"],
            "destination": os.environ["INT_SPECS_DIR"]
        },
    ]
}

# Writing the mounts file.
with open(mounts_file, "w") as mfile:
    json.dump(drive_map, mfile, indent=4)

In [3]:
!cat ~/.tao_mounts.json

{
    "Mounts": [
        {
            "source": "/home/pepeleduin/mobia/mobia-project/training/lpd_net",
            "destination": "/workspace/tao-experiments/lpd_net"
        },
        {
            "source": "/home/pepeleduin/mobia/mobia-project/training/lpd_net/specs",
            "destination": "/workspace/tao-experiments/lpd_net/specs"
        }
    ]
}

## 1. Install the requirements and the TAO launcher <a class="anchor" id="head-1"></a>

_Note: Please, ensure to follow the instructions on [README.md](README.md) before continue._

TAO Toolkit recommends users to run the TAO launcher in a virtual env with Python version between 3.6.9 and 3.8. In this project, we use Anaconda to manage the virtual environment and we need to the NVIDIA Container Toolkit.

### 1.1 Installing on Anaconda <a class="anchor" id="head-1-1"></a>
We consider that you already activate the environment with Conda.

Requirements:
* Python version >=3.6.9 < 3.8 (Already Installed)
* Jupyter Lab (Already Installed)
* OpenCV
* Pillow
* Pip (Already Installed)

In [4]:
!pip install opencv-python
!pip install Pillow

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


### 1.2 Install [NVIDIA Container Toolkit](https://docs.nvidia.com/datacenter/cloud-native/container-toolkit/install-guide.html) <a class="anchor" id="head-1-2"></a>
Requirements:
* nvidia-container-toolkit > 1.3.0-1
* nvidia-container-runtime > 3.4.0-1
* nvidia-docker2 > 2.5.0-1
* nvidia-driver > 455

In [5]:
# Check Requirements
!python --version 
!docker --version
!dpkg -l | grep -e nvidia-container-toolkit -e nvidia-docker
!nvidia-smi | grep CUDA

Python 3.7.13
Docker version 20.10.14, build a224086
ii  nvidia-container-toolkit              1.9.0-1                          amd64        NVIDIA container runtime hook
ii  nvidia-docker2                        2.10.0-1                         all          nvidia-docker CLI wrapper
| NVIDIA-SMI 470.103.01   Driver Version: 470.103.01   CUDA Version: 11.4     |


### 1.3 Install Tao Launcher <a class="anchor" id="head-1-3"></a>
The TAO launcher is a python package distributed as a Python wheel listed in the `nvidia-pyindex` Python index. You may install the Tao launcher by executing the following cell.

In [14]:
# SKIP this step IF you have already installed the TAO launcher wheel.
!pip install nvidia-pyindex
!pip install nvidia-tao

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [6]:
# View the versions of the TAO launcher
!tao info

Configuration of the TAO Toolkit Instance
dockers: ['nvidia/tao/tao-toolkit-tf', 'nvidia/tao/tao-toolkit-pyt', 'nvidia/tao/tao-toolkit-lm']
format_version: 2.0
toolkit_version: 3.22.02
published_date: 02/28/2022


## 2. Prepare dataset and pre-trained model <a class="anchor" id="head-2"></a>

### 2.A Download the dataset <a class="anchor" id="head-2-1"></a>
First, sync the [OpenALPR benchmarks](https://github.com/openalpr/benchmarks) inside `$LOCAL_DATA_DIR`:

In [7]:
!git clone https://github.com/openalpr/benchmarks $LOCAL_DATA_DIR/benchmarks

Cloning into '/home/pepeleduin/mobia/mobia-project/training/lpd_net/data/benchmarks'...
remote: Enumerating objects: 1752, done.
remote: Total 1752 (delta 0), reused 0 (delta 0), pack-reused 1752
Receiving objects: 100% (1752/1752), 187.98 MiB | 4.01 MiB/s, done.
Resolving deltas: 100% (34/34), done.


### 2.B Resize images and labels <a class="anchor" id="head-2-2"></a>
Next, use the script `lpd_prepare_data.py` and run the following command to resize images/labels with width=640 and height=480 located in `$LOCAL_DATA_DIR/benchmarks/endtoend/us`. After this step, the data is organized in `$LOCAL_DATA_DIR/lpd` as follows:
```
└──dataset root
  ├── images
  |   ├── 000000.jpg
  |   ├── 000001.jpg
  |   |       .
  |   |       .
  |   └── xxxxxx.jpg
  ├── labels
  |   ├── 000000.txt
  |   ├── 000001.txt
  |   |       .
  |   |       .
  |   └── xxxxxx.txt
  └── kitti_seq_to_map.json
```

`kitti_seq_to_map.json` is optional.

In [8]:
!python lpd_prepare_data.py --input_dir $LOCAL_DATA_DIR/benchmarks/endtoend/us \
                            --output_dir $LOCAL_DATA_DIR/lpd \
                            --target_width  640 \
                            --target_height 480

Resizing /home/pepeleduin/mobia/mobia-project/training/lpd_net/data/benchmarks/endtoend/us/wts-lg-000155.jpg and its label
Resizing /home/pepeleduin/mobia/mobia-project/training/lpd_net/data/benchmarks/endtoend/us/cfaa9dd2-a388-4e92-bb3a-ae65c28d8139.jpg and its label
Resizing /home/pepeleduin/mobia/mobia-project/training/lpd_net/data/benchmarks/endtoend/us/wts-lg-000165.jpg and its label
Resizing /home/pepeleduin/mobia/mobia-project/training/lpd_net/data/benchmarks/endtoend/us/car8.jpg and its label
Resizing /home/pepeleduin/mobia/mobia-project/training/lpd_net/data/benchmarks/endtoend/us/wts-lg-000170.jpg and its label
Resizing /home/pepeleduin/mobia/mobia-project/training/lpd_net/data/benchmarks/endtoend/us/wts-lg-000023.jpg and its label
Resizing /home/pepeleduin/mobia/mobia-project/training/lpd_net/data/benchmarks/endtoend/us/wts-lg-000043.jpg and its label
Resizing /home/pepeleduin/mobia/mobia-project/training/lpd_net/data/benchmarks/endtoend/us/wts-lg-000151.jpg and its label
Re

### 2.C Split data <a class="anchor" id="head-2-3"></a>

A spec file called `SPECS_tfrecord.txt` located in `$INT_SPECS_DIR` splits the data. It points to the same path in local machine, but internal to Tao.

In [9]:
!cat $LOCAL_SPECS_DIR/SPECS_tfrecord.txt

kitti_config {
  root_directory_path: "/workspace/tao-experiments/lpd_net/data/lpd/data"
  image_dir_name: "image"
  label_dir_name: "label"
  image_extension: ".jpg"
  partition_mode: "random"
  num_partitions: 2
  val_split: 20
  num_shards: 4
}

image_directory_path: "/workspace/tao-experiments/lpd_net/data/lpd/data"

The following paths must be changed in `SPECS_tfrecord.txt`: `root_directory_path`, `image_directory_path`. These paths should contain the output of the cell below:

In [10]:
!echo $INT_DATA_DIR/lpd/data/

/workspace/tao-experiments/lpd_net/data/lpd/data/


 Later, split the data into two parts: 80% for the training set and 20% for the validation set. Run the following command to split the dataset randomly and generate tfrecords. The first that you will use `tao`, the command will start to download the `nvcr.io/nvidia/tao/tao-toolkit-tf` image.

In [11]:
!tao detectnet_v2 dataset_convert \
                -d $INT_SPECS_DIR/SPECS_tfrecord.txt \
                -o $INT_DATA_DIR/lpd_tfrecord/lpd

2022-04-08 10:37:00,497 [INFO] root: Registry: ['nvcr.io']
2022-04-08 10:37:00,561 [INFO] tlt.components.instance_handler.local_instance: Running command in container: nvcr.io/nvidia/tao/tao-toolkit-tf:v3.21.11-tf1.15.4-py3
2022-04-08 10:37:00,604 [WARNING] tlt.components.docker_handler.docker_handler: 
Docker will run the commands as root. If you would like to retain your
local host permissions, please add the "user":"UID:GID" in the
DockerOptions portion of the "/home/pepeleduin/.tao_mounts.json" file. You can obtain your
users UID and GID by using the "id -u" and "id -g" commands on the
terminal.
Using TensorFlow backend.
Using TensorFlow backend.
2022-04-08 15:37:06,956 [INFO] iva.detectnet_v2.dataio.build_converter: Instantiating a kitti converter
2022-04-08 15:37:06,956 [INFO] root: Instantiating a kitti converter
2022-04-08 15:37:06,956 [INFO] iva.detectnet_v2.dataio.dataset_converter_lib: Creating output directory /workspace/tao-experiments/lpd_net/data/lpd_tfrecord
2022-04-08 

### 2.D Download pre-trained model <a class="anchor" id="head-2-4"></a>
Download the correct pretrained model from the NGC model registry for your experiment. Please note that for DetectNet_v2, the input is expected to be 0-1 normalized with input channels in RGB order. Therefore, for optimum results please download model templates from `nvidia/tao/pretrained_detectnet_v2`. The templates are now organized as version strings. For example, to download a resnet18 model suitable for detectnet please resolve to the ngc object shown as `nvidia/tao/pretrained_detectnet_v2:resnet18`. 

All other models are in BGR order expect input preprocessing with mean subtraction and input channels. Using them as pretrained weights may result in suboptimal performance.

You may also use this notebook with the following purpose-built pretrained models 
* [PeopleNet](https://ngc.nvidia.com/catalog/models/nvidia:tao:peoplenet)
* [TrafficCamNet](https://ngc.nvidia.com/catalog/models/nvidia:tao:trafficcamnet)
* [DashCamNet](https://ngc.nvidia.com/catalog/models/nvidia:tao:dashcamnet)
* [FaceDetect-IR](https://ngc.nvidia.com/catalog/models/nvidia:tao:facedetectir) 

In [12]:
# Installing NGC CLI on the local machine.
## Download and install
%env CLI=ngccli_cat_linux.zip
!mkdir -p $LOCAL_PROJECT_DIR/ngccli

# Remove any previously existing CLI installations
!rm -rf $LOCAL_PROJECT_DIR/ngccli/*
!wget "https://ngc.nvidia.com/downloads/$CLI" -P $LOCAL_PROJECT_DIR/ngccli
!unzip  -u "$LOCAL_PROJECT_DIR/ngccli/$CLI" \
        -d $LOCAL_PROJECT_DIR/ngccli/
!rm $LOCAL_PROJECT_DIR/ngccli/*.zip 
os.environ["PATH"]="{}/ngccli:{}".format(os.getenv("LOCAL_PROJECT_DIR", ""),
                                         os.getenv("PATH", ""))

env: CLI=ngccli_cat_linux.zip
--2022-04-08 10:37:33--  https://ngc.nvidia.com/downloads/ngccli_cat_linux.zip
Resolving ngc.nvidia.com (ngc.nvidia.com)... 18.64.174.22, 18.64.174.116, 18.64.174.58, ...
Connecting to ngc.nvidia.com (ngc.nvidia.com)|18.64.174.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 32588469 (31M) [application/zip]
Saving to: ‘/home/pepeleduin/mobia/mobia-project/training/lpd_net/ngccli/ngccli_cat_linux.zip’

ngccli_cat_linux.zi 100%[===================>]  31.08M  4.21MB/s    in 7.6s    

2022-04-08 10:37:41 (4.08 MB/s) - ‘/home/pepeleduin/mobia/mobia-project/training/lpd_net/ngccli/ngccli_cat_linux.zip’ saved [32588469/32588469]

Archive:  /home/pepeleduin/mobia/mobia-project/training/lpd_net/ngccli/ngccli_cat_linux.zip
  inflating: /home/pepeleduin/mobia/mobia-project/training/lpd_net/ngccli/ngc  
 extracting: /home/pepeleduin/mobia/mobia-project/training/lpd_net/ngccli/ngc.md5  


In [13]:
# List models available in the model registry.
!ngc registry model list nvidia/tao/lpdnet:*

+-------+-------+-------+-------+-------+-------+-------+-------+-------+
| Versi | Accur | Epoch | Batch | GPU   | Memor | File  | Statu | Creat |
| on    | acy   | s     | Size  | Model | y Foo | Size  | s     | ed    |
|       |       |       |       |       | tprin |       |       | Date  |
|       |       |       |       |       | t     |       |       |       |
+-------+-------+-------+-------+-------+-------+-------+-------+-------+
| unpru | 98.0  | 120   | 1     | V100  | 67.5  | 67.53 | UPLOA | Nov   |
| ned_v |       |       |       |       |       | MB    | D_COM | 24,   |
| 2.0   |       |       |       |       |       |       | PLETE | 2021  |
| unpru | 98.0  | 120   | 1     | V100  | 85.8  | 85.83 | UPLOA | Aug   |
| ned_v |       |       |       |       |       | MB    | D_COM | 24,   |
| 1.0   |       |       |       |       |       |       | PLETE | 2021  |
| prune | 98.0  | 120   | 1     | V100  | 5.3   | 5.34  | UPLOA | Nov   |
| d_v2. |       |       |       |     

In [14]:
!mkdir -p $LOCAL_EXPERIMENT_DIR/

In [15]:
# Download the pretrained model from NGC
!ngc registry model download-version nvidia/tao/lpdnet:unpruned_v1.0 \
    --dest $LOCAL_EXPERIMENT_DIR/

Downloaded 85.86 MB in 29s, Download speed: 2.96 MB/s               
----------------------------------------------------
Transfer id: lpdnet_vunpruned_v1.0 Download status: Completed.
Downloaded local path: /home/pepeleduin/mobia/mobia-project/training/lpd_net/experiments/lpdnet_vunpruned_v1.0
Total files downloaded: 2 
Total downloaded size: 85.86 MB
Started at: 2022-04-08 10:37:57.303291
Completed at: 2022-04-08 10:38:26.335331
Duration taken: 29s
----------------------------------------------------


In [16]:
# Check the download model status
!ls -rlt $LOCAL_EXPERIMENT_DIR/

total 4
drwx------ 2 pepeleduin pepeleduin 4096 Apr  8 10:38 lpdnet_vunpruned_v1.0


## 3. Provide training specification <a class="anchor" id="head-3"></a>
* Tfrecords for the train datasets
    * To use the newly generated tfrecords, update the dataset_config parameter in the spec file at `$SPECS_DIR/SPECS_train.txt` 
    * Update the fold number to use for evaluation. In case of random data split, please use fold `0` only
    * For sequence-wise split, you may use any fold generated from the dataset convert tool
* Pre-trained models
* Augmentation parameters for on the fly data augmentation
* Other training (hyper-)parameters such as batch size, number of epochs, learning rate etc.

In [21]:
!cat $LOCAL_SPECS_DIR/SPECS_train.txt

################################################################################
# The MIT License (MIT)
#
# Copyright (c) 2019-2021 NVIDIA CORPORATION
#
# Permission is hereby granted, free of charge, to any person obtaining a
# copy of this software and associated documentation files (the "Software"),
# to deal in the Software without restriction, including without limitation
# the rights to use, copy, modify, merge, publish, distribute, sublicense,
# and/or sell copies of the Software, and to permit persons to whom the
# Software is furnished to do so, subject to the following conditions:
#
# The above copyright notice and this permission notice shall be included in
# all copies or substantial portions of the Software.
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT.  IN NO EVENT SHALL
# THE AUTHORS OR COPYRIGHT HOLDERS BE L

## 4. Run TAO training <a class="anchor" id="head-4"></a>
* Provide the sample spec file and the output directory location for models

*Note: The training may take hours to complete. Also, the remaining notebook, assumes that the training was done in single-GPU mode. When run in multi-GPU mode, please expect to update the pruning and inference steps with new pruning thresholds and updated parameters in the clusterfile.json accordingly for optimum performance.*

*Detectnet_v2 now supports restart from checkpoint. In case the training job is killed prematurely, you may resume training from the closest checkpoint by simply re-running the **same** command line. Please do make sure to use the <u>**same number of GPUs**</u> when restarting the training.*

*When running the training with NUM_GPUs>1, you may need to modify the `batch_size_per_gpu` and `learning_rate` to get similar mAP as a 1GPU training run. In most cases, scaling down the batch-size by a factor of NUM_GPU's or scaling up the learning rate by a factor of NUM_GPU's would be a good place to start.*

Useful [link](https://docs.nvidia.com/tao/tao-toolkit/text/object_detection/detectnet_v2.html) to documentation for Tao Detectnet V2.

In [17]:
# Set a env variable to define the name of the model
%env MODEL_NAME=lpd_net_test
%env LOG_NAME=log_lpd_net_test01

env: MODEL_NAME=lpd_net_test
env: LOG_NAME=log_lpd_net_test01


In [19]:
!tao detectnet_v2 train -e $INT_SPECS_DIR/SPECS_train.txt \
                        -r $INT_MAIN_DIR/experiments/dir_$MODEL_NAME \
                        -k nvidia_tlt \
                        -n $MODEL_NAME
#\
                        #--log_file $INT_MAIN_DIR/experiments/dir_$MODEL_NAME/$LOG_NAME

2022-04-08 10:45:02,664 [INFO] root: Registry: ['nvcr.io']
2022-04-08 10:45:02,727 [INFO] tlt.components.instance_handler.local_instance: Running command in container: nvcr.io/nvidia/tao/tao-toolkit-tf:v3.21.11-tf1.15.4-py3
2022-04-08 10:45:02,754 [WARNING] tlt.components.docker_handler.docker_handler: 
Docker will run the commands as root. If you would like to retain your
local host permissions, please add the "user":"UID:GID" in the
DockerOptions portion of the "/home/pepeleduin/.tao_mounts.json" file. You can obtain your
users UID and GID by using the "id -u" and "id -g" commands on the
terminal.
Using TensorFlow backend.
Using TensorFlow backend.

2022-04-08 15:45:08,106 [WARNING] tensorflow: From /opt/tlt/.cache/dazel/_dazel_tlt/75913d2aee35770fa76c4a63d877f3aa/execroot/ai_infra/bazel-out/k8-fastbuild/bin/magnet/packages/iva/build_wheel.runfiles/ai_infra/iva/detectnet_v2/cost_function/cost_auto_weight_hook.py:43: The name tf.train.SessionRunHook is deprecated. Please use tf.estima

In [22]:
!cat ./experiments/dir_lpd_net_test/status.json

{"date": "4/8/2022", "time": "15:45:8", "status": "Starting DetectNet_v2 Training job"}
{"date": "4/8/2022", "time": "15:45:8", "status": "Training gridbox model."}
{"date": "4/8/2022", "time": "15:45:8", "status": "Building DetectNet V2 model"}
{"date": "4/8/2022", "time": "15:45:17", "status": "DetectNet V2 model built."}
{"date": "4/8/2022", "time": "15:45:17", "status": "Building rasterizer."}
{"date": "4/8/2022", "time": "15:45:17", "status": "Rasterizers built."}
{"date": "4/8/2022", "time": "15:45:17", "status": "Building training graph."}
{"date": "4/8/2022", "time": "15:45:17", "status": "Rasterizing tensors."}
{"date": "4/8/2022", "time": "15:45:17", "status": "Tensors rasterized."}
{"date": "4/8/2022", "time": "15:45:19", "status": "Training graph built."}
{"date": "4/8/2022", "time": "15:45:19", "status": "Building validation graph."}
{"date": "4/8/2022", "time": "15:45:19", "status": "Rasterizing tensors."}
{"date": "4/8/2022", "time": "15:45:20", "status": "Tensors raster

In [ ]:
!tao detectnet_v2 train -e $SPECS_DIR/detectnet_v2_train_resnet18_kitti.txt \
                        -r $USER_EXPERIMENT_DIR/experiment_dir_unpruned \
                        -k $KEY \
                        -n resnet18_detector \
                        --gpus $NUM_GPUS

In [59]:
print('Model for each epoch:')
print('---------------------')
!ls -lh $LOCAL_PROJECT_DIR/experiments/experiment_test/weights

Model for each epoch:
---------------------
total 0


## 5. Evaluate the trained model <a class="anchor" id="head-5"></a>

In [ ]:
!tao detectnet_v2 evaluate -e $SPECS_DIR/detectnet_v2_train_resnet18_kitti.txt\
                           -m $INT_MAIN_DIR/experiment_dir_unpruned/weights/resnet18_detector.tlt \
                           -k $KEY

## 6. Prune the trained model <a class="anchor" id="head-6"></a>
* Specify pre-trained model
* Equalization criterion (`Applicable for resnets and mobilenets`)
* Threshold for pruning.
* A key to save and load the model
* Output directory to store the model

*Usually, you just need to adjust `-pth` (threshold) for accuracy and model size trade off. Higher `pth` gives you smaller model (and thus higher inference speed) but worse accuracy. The threshold to use is dependent on the dataset. A pth value `5.2e-6` is just a start point. If the retrain accuracy is good, you can increase this value to get smaller models. Otherwise, lower this value to get better accuracy.*

*For some internal studies, we have noticed that a pth value of 0.01 is a good starting point for detectnet_v2 models.*

In [ ]:
# Create an output directory if it doesn't exist.
!mkdir -p $LOCAL_EXPERIMENT_DIR/experiment_dir_pruned

In [ ]:
!tao detectnet_v2 prune \
                  -m $INT_MAIN_DIR/experiment_dir_unpruned/weights/resnet18_detector.tlt \
                  -o $INT_MAIN_DIR/experiment_dir_pruned/resnet18_nopool_bn_detectnet_v2_pruned.tlt \
                  -eq union \
                  -pth 0.0000052 \
                  -k $KEY

In [ ]:
!ls -rlt $LOCAL_EXPERIMENT_DIR/experiment_dir_pruned/

## 7. Retrain the pruned model <a class="anchor" id="head-7"></a>
* Model needs to be re-trained to bring back accuracy after pruning
* Specify re-training specification with pretrained weights as pruned model.

*Note: For retraining, please set the `load_graph` option to `true` in the model_config to load the pruned model graph. Also, if after retraining, the model shows some decrease in mAP, it could be that the originally trained model was pruned a little too much. Please try reducing the pruning threshold (thereby reducing the pruning ratio) and use the new model to retrain.*

*Note: DetectNet_v2 now supports Quantization Aware Training, to help with optmizing the model. By default, the training in the cell below doesn't run the model with QAT enabled. For information on training a model with QAT, please refer to the cells under [section 11](#head-11)*

In [ ]:
# Printing the retrain experiment file. 
# Note: We have updated the experiment file to include the 
# newly pruned model as a pretrained weights and, the
# load_graph option is set to true 
!cat $LOCAL_SPECS_DIR/detectnet_v2_retrain_resnet18_kitti.txt

In [ ]:
# Retraining using the pruned model as pretrained weights 
!tao detectnet_v2 train -e $SPECS_DIR/detectnet_v2_retrain_resnet18_kitti.txt \
                        -r $USER_EXPERIMENT_DIR/experiment_dir_retrain \
                        -k $KEY \
                        -n resnet18_detector_pruned \
                        --gpus $NUM_GPUS

In [ ]:
# Listing the newly retrained model.
!ls -rlt $LOCAL_EXPERIMENT_DIR/experiment_dir_retrain/weights

## 8. Evaluate the retrained model <a class="anchor" id="head-8"></a>

This section evaluates the pruned and retrained model, using the `evaluate` command.

In [ ]:
!tao detectnet_v2 evaluate -e $SPECS_DIR/detectnet_v2_retrain_resnet18_kitti.txt \
                           -m $INT_MAIN_DIR/experiment_dir_retrain/weights/resnet18_detector_pruned.tlt \
                           -k $KEY

## 9. Visualize inferences <a class="anchor" id="head-9"></a>
In this section, we run the `inference` tool to generate inferences on the trained models. To render bboxes from more classes, please edit the spec file `detectnet_v2_inference_kitti_tlt.txt` to include all the classes you would like to visualize and edit the rest of the file accordingly.

In [ ]:
# Running inference for detection on n images
!tao detectnet_v2 inference -e $SPECS_DIR/detectnet_v2_inference_kitti_tlt.txt \
                            -o $USER_EXPERIMENT_DIR/tlt_infer_testing \
                            -i $INT_DATA_DIR/testing/image_2 \
                            -k $KEY

The `inference` tool produces two outputs. 
1. Overlain images in `$INT_MAIN_DIR/tlt_infer_testing/images_annotated`
2. Frame by frame bbox labels in kitti format located in `$INT_MAIN_DIR/tlt_infer_testing/labels`

*Note: To run inferences for a single image, simply replace the path to the -i flag in `inference` command with the path to the image.*

In [ ]:
# Simple grid visualizer
!pip3 install matplotlib==3.3.3
%matplotlib inline
import matplotlib.pyplot as plt
import os
from math import ceil
valid_image_ext = ['.jpg', '.png', '.jpeg', '.ppm']

def visualize_images(image_dir, num_cols=4, num_images=10):
    output_path = os.path.join(os.environ['LOCAL_EXPERIMENT_DIR'], image_dir)
    num_rows = int(ceil(float(num_images) / float(num_cols)))
    f, axarr = plt.subplots(num_rows, num_cols, figsize=[80,30])
    f.tight_layout()
    a = [os.path.join(output_path, image) for image in os.listdir(output_path) 
         if os.path.splitext(image)[1].lower() in valid_image_ext]
    for idx, img_path in enumerate(a[:num_images]):
        col_id = idx % num_cols
        row_id = idx // num_cols
        img = plt.imread(img_path)
        axarr[row_id, col_id].imshow(img) 

In [ ]:
# Visualizing the first 12 images.
OUTPUT_PATH = 'tlt_infer_testing/images_annotated' # relative path from $INT_MAIN_DIR.
COLS = 4 # number of columns in the visualizer grid.
IMAGES = 12 # number of images to visualize.

visualize_images(OUTPUT_PATH, num_cols=COLS, num_images=IMAGES)

## 10. Model Export <a class="anchor" id="head-10"></a>

In [ ]:
!mkdir -p $LOCAL_EXPERIMENT_DIR/experiment_dir_final
# Removing a pre-existing copy of the etlt if there has been any.
import os
output_file=os.path.join(os.environ['LOCAL_EXPERIMENT_DIR'],
                         "experiment_dir_final/resnet18_detector.etlt")
if os.path.exists(output_file):
    os.system("rm {}".format(output_file))
!tao detectnet_v2 export \
                  -m $USER_EXPERIMENT_DIR/experiment_dir_retrain/weights/resnet18_detector_pruned.tlt \
                  -o $INT_MAIN_DIR/experiment_dir_final/resnet18_detector.etlt \
                  -k $KEY

In [ ]:
print('Exported model:')
print('------------')
!ls -lh $LOCAL_EXPERIMENT_DIR/experiment_dir_final

### A. Int8 Optimization <a class="anchor" id="head-10-1"></a>
DetectNet_v2 model supports int8 inference mode in TensorRT. 
In order to use int8 mode, we must calibrate the model to run 8-bit inferences -

* Generate calibration tensorfile from the training data using detectnet_v2 calibration_tensorfile
* Use tao <task> export to generate int8 calibration table.

*Note: For this example, we generate a calibration tensorfile containing 10 batches of training data.
Ideally, it is best to use at least 10-20% of the training data to do so. The more data provided during calibration, the closer int8 inferences are to fp32 inferences.*

*Note: If the model was trained with QAT nodes available, please refrain from using the post training int8 optimization as mentioned below. Please export the model in int8 mode (using the arg `--data_type int8`) with just the path to the calibration cache file (using the argument `--cal_cache_file`)*

In [ ]:
!tao detectnet_v2 calibration_tensorfile -e $SPECS_DIR/detectnet_v2_retrain_resnet18_kitti.txt \
                                         -m 10 \
                                         -o $INT_MAIN_DIR/experiment_dir_final/calibration.tensor

In [ ]:
!rm -rf $LOCAL_EXPERIMENT_DIR/experiment_dir_final/resnet18_detector.etlt
!rm -rf $LOCAL_EXPERIMENT_DIR/experiment_dir_final/calibration.bin
!tao detectnet_v2 export \
                  -m $INT_MAIN_DIR/experiment_dir_retrain/weights/resnet18_detector_pruned.tlt \
                  -o $USER_EXPERIMENT_DIR/experiment_dir_final/resnet18_detector.etlt \
                  -k $KEY  \
                  --cal_data_file $INT_MAIN_DIR/experiment_dir_final/calibration.tensor \
                  --data_type int8 \
                  --batches 10 \
                  --batch_size 4 \
                  --max_batch_size 4\
                  --engine_file $INT_MAIN_DIR/experiment_dir_final/resnet18_detector.trt.int8 \
                  --cal_cache_file $INT_MAIN_DIR/experiment_dir_final/calibration.bin \
                  --verbose

### B. Generate TensorRT engine <a class="anchor" id="head-10-2"></a>
Verify engine generation using the `tao-converter` utility included with the docker.

The `tao-converter` produces optimized tensorrt engines for the platform that it resides on. Therefore, to get maximum performance, please instantiate this docker and execute the `tao-converter` command, with the exported `.etlt` file and calibration cache (for int8 mode) on your target device. The tao-converter utility included in this docker only works for x86 devices, with discrete NVIDIA GPU's. 

For the jetson devices, please download the tao-converter for jetson from the dev zone link [here](https://developer.nvidia.com/tao-converter). 

If you choose to integrate your model into deepstream directly, you may do so by simply copying the exported `.etlt` file along with the calibration cache to the target device and updating the spec file that configures the `gst-nvinfer` element to point to this newly exported model. Usually this file is called `config_infer_primary.txt` for detection models and `config_infer_secondary_*.txt` for classification models.

In [ ]:
!tao converter $USER_EXPERIMENT_DIR/experiment_dir_final/resnet18_detector.etlt \
                   -k $KEY \
                    -c $INT_MAIN_DIR/experiment_dir_final/calibration.bin \
                   -o output_cov/Sigmoid,output_bbox/BiasAdd \
                   -d 3,384,1248 \
                   -i nchw \
                   -m 64 \
                   -t int8 \
                   -e $USER_EXPERIMENT_DIR/experiment_dir_final/resnet18_detector.trt \
                   -b 4

## 11. Verify Deployed Model <a class="anchor" id="head-11"></a>
Verify the exported model by visualizing inferences on TensorRT.
In addition to running inference on a `.tlt` model in [step 9](#head-9), the `inference` tool is also capable of consuming the converted `TensorRT engine` from [step 10.B](#head-10-2).

*If after int-8 calibration the accuracy of the int-8 inferences seem to degrade, it could be because the there wasn't enough data in the calibration tensorfile used to calibrate thee model or, the training data is not entirely representative of your test images, and the calibration maybe incorrect. Therefore, you may either regenerate the calibration tensorfile with more batches of the training data and recalibrate the model, or calibrate the model on a few images from the test set. This may be done using `--cal_image_dir` flag in the `export` tool. For more information, please follow the instructions in the USER GUIDE.

### A. Inference using TensorRT engine <a class="anchor" id="head-11-1"></a>

In [ ]:
!tao detectnet_v2 inference -e $SPECS_DIR/detectnet_v2_inference_kitti_etlt.txt \
                            -o $INT_MAIN_DIR/etlt_infer_testing \
                            -i $INT_DATA_DIR/testing/image_2 \
                            -k $KEY

In [ ]:
# visualize the first 12 inferenced images.
OUTPUT_PATH = 'etlt_infer_testing/images_annotated' # relative path from $USER_EXPERIMENT_DIR.
COLS = 4 # number of columns in the visualizer grid.
IMAGES = 12 # number of images to visualize.

visualize_images(OUTPUT_PATH, num_cols=COLS, num_images=IMAGES)

## 11. QAT workflow <a class="anchor" id="head-12"></a>
This section delves into the newly enabled Quantization Aware Training feature with DetectNet_v2. The workflow defined below converts a pruned model from section [5](#head-5) to enable QAT and retrain this model to while accounting the noise introduced due to quantization in the forward pass. 

### A. Convert pruned model to QAT and retrain <a class="anchor" id="head-12-1"></a>
All detectnet models, unpruned and pruned models can be converted to QAT models by setting the `enable_qat` parameter in the `training_config` component of the spec file to `true`.

In [ ]:
# Printing the retrain experiment file. 
# Note: We have updated the experiment file to convert the
# pretrained model to qat mode by setting the enable_qat
# parameter.
!cat $LOCAL_SPECS_DIR/detectnet_v2_retrain_resnet18_kitti_qat.txt

In [ ]:
!tao detectnet_v2 train -e $SPECS_DIR/detectnet_v2_retrain_resnet18_kitti_qat.txt \
                        -r $INT_MAIN_DIR/experiment_dir_retrain_qat \
                        -k $KEY \
                        -n resnet18_detector_pruned_qat \
                        --gpus $NUM_GPUS

In [ ]:
!ls -rlt $LOCAL_EXPERIMENT_DIR/experiment_dir_retrain_qat/weights

### B. Evaluate QAT converted model <a class="anchor" id="head-12-2"></a>
This section evaluates a QAT enabled pruned retrained model. The mAP of this model should be comparable to that of the pruned retrained model without QAT. However, due to quantization, it is possible sometimes to see a drop in the mAP value for certain datasets.

In [ ]:
!tao detectnet_v2 evaluate -e $SPECS_DIR/detectnet_v2_retrain_resnet18_kitti_qat.txt \
                           -m $USER_EXPERIMENT_DIR/experiment_dir_retrain_qat/weights/resnet18_detector_pruned_qat.tlt \
                           -k $KEY \
                           -f tlt

### C. Export QAT trained model to int8 <a class="anchor" id="head-12-3"></a>
Export a QAT trained model to TensorRT parsable model. This command generates an .etlt file from the trained model and the serializes corresponding int8 scales as a TRT readable calibration cache file.

In [ ]:
!rm -rf $LOCAL_EXPERIMENT_DIR/experiment_dir_final/resnet18_detector_qat.etlt
!rm -rf $LOCAL_EXPERIMENT_DIR/experiment_dir_final/calibration_qat.bin
!tao detectnet_v2 export \
                  -m $INT_MAIN_DIR/experiment_dir_retrain_qat/weights/resnet18_detector_pruned_qat.tlt \
                  -o $USER_EXPERIMENT_DIR/experiment_dir_final/resnet18_detector_qat.etlt \
                  -k $KEY  \
                  --data_type int8 \
                  --batch_size 64 \
                  --max_batch_size 64\
                  --engine_file $INT_MAIN_DIR/experiment_dir_final/resnet18_detector_qat.trt.int8 \
                  --cal_cache_file $USER_EXPERIMENT_DIR/experiment_dir_final/calibration_qat.bin \
                  --verbose

### D. Evaluate a QAT trained model using the exported TensorRT engine <a class="anchor" id="head-12-4"></a>
This section evaluates a QAT enabled pruned retrained model using the TensorRT int8 engine that was exported in [Section C](#head-12-3). Please note that there maybe a slight difference (~0.1-0.5%) in the mAP from [Section B](#head-12-2), oweing to some differences in the implementation of quantization in TensorRT.

*Note: The TensorRT evaluator might be slightly slower than the TAO evaluator here, because the evaluation dataloader is pinned to the CPU to avoid any clashes between TensorRT and TAO instances in the GPU. Please note that this tool was not intended and has not been developed for profiling the model. It is just a means to qualitatively analyse the model.*

*Please use native TensorRT or DeepStream for the most optimized inferences.*

In [ ]:
!tao detectnet_v2 evaluate -e $SPECS_DIR/detectnet_v2_retrain_resnet18_kitti_qat.txt \
                           -m $INT_MAIN_DIR/experiment_dir_final/resnet18_detector_qat.trt.int8 \
                           -f tensorrt

### E. Inference using QAT engine <a class="anchor" id="head-12-5"></a>
Run inference and visualize detections on test images, using the exported TensorRT engine from [Section C](#head-12-3).

In [ ]:
!tao detectnet_v2 inference -e $SPECS_DIR/detectnet_v2_inference_kitti_etlt_qat.txt \
                            -o $USER_EXPERIMENT_DIR/tlt_infer_testing_qat \
                            -i $INT_DATA_DIR/testing/image_2 \
                            -k $KEY

In [ ]:
# visualize the first 12 inferenced images.
OUTPUT_PATH = 'tlt_infer_testing_qat/images_annotated' # relative path from $USER_EXPERIMENT_DIR.
COLS = 4 # number of columns in the visualizer grid.
IMAGES = 12 # number of images to visualize.

visualize_images(OUTPUT_PATH, num_cols=COLS, num_images=IMAGES)